In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression


Bad key "text.kerning_factor" on line 4 in
C:\Users\Sean\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


# Regression Analysis: Seasonal Effects with Sklearn Linear Regression
In this notebook, you will build a SKLearn linear regression model to predict Yen futures ("settle") returns with *lagged* Yen futures returns. 

In [2]:
# Futures contract on the Yen-dollar exchange rate:
# This is the continuous chain of the futures contracts that are 1 month to expiration
yen_futures = pd.read_csv(
    Path("yen.csv"), index_col="Date", infer_datetime_format=True, parse_dates=True
)
yen_futures.head()

FileNotFoundError: [Errno 2] File b'yen.csv' does not exist: b'yen.csv'

In [ ]:
# Trim the dataset to begin on January 1st, 1990
yen_futures = yen_futures.loc["1990-01-01":, :]
yen_futures.head()

# Data Preparation

### Returns

In [ ]:
yen_futures['Returns'] = yen_futures['Settle'].pct_change() * 100
yen_futures = yen_futures.replace(-np.inf, np.nan).dropna()
yen_futures.tail()


In [ ]:
# Create a series using "Settle" price percentage returns, drop any nan"s, and check the results:
# (Make sure to multiply the pct_change() results by 100)
# In this case, you may have to replace inf, -inf values with np.nan"s
# YOUR CODE HERE!

### Lagged Returns 

In [ ]:
# Create a lagged return using the shift function
# YOUR CODE HERE!

yen_futures['Lagged_Returns'] = yen_futures['Returns'].shift()

# Drop NaN values
yen_futures = yen_futures.dropna()
yen_futures.tail()


### Train Test Split

In [ ]:
# Create a train/test split for the data using 2018-2019 for testing and the rest for training
train = yen_futures[:'2017']
test = yen_futures['2018':]



In [ ]:
# Create four dataframes:
# X_train (training set using just the independent variables), X_test (test set of of just the independent variables)
# Y_train (training set using just the "y" variable, i.e., "Futures Return"), Y_test (test set of just the "y" variable):
# YOUR CODE HERE!

X_train = train['Lagged_Returns'].values.reshape(-1,1) 


Y_train = train['Returns']


X_test = test['Lagged_Returns'].values.reshape(-1,1)

Y_test = test['Returns']




In [ ]:
X_train

# Linear Regression Model

In [ ]:
# Create a Linear Regression model and fit it to the training data


# Fit a SKLearn linear regression using just the training set (X_train, Y_train):
# YOUR CODE HERE!

model_1 = LinearRegression()


model_1.fit(X_train, Y_train)

print(model_1.coef_)
print(model_1.intercept_)

predicted_y_train = model_1.predict(X_train)


# Make predictions using the Testing Data

Note: We want to evaluate the model using data that it has never seen before, in this case: X_test.

In [ ]:
# Make a prediction of "y" values using just the test dataset
# YOUR CODE HERE!

predicted_y_test = model_1.predict(X_test)



In [ ]:
# Assemble actual y data (Y_test) with predicted y data (from just above) into two columns in a dataframe:
# YOUR CODE HERE!
Results = Y_test.to_frame()
Results["Predicted Return"] = predicted_y_test

Results.head()

In [ ]:
# Plot the first 20 predictions vs the true values
# YOUR CODE HERE!
Results[:20].plot(subplots=True)

# Out-of-Sample Performance

Evaluate the model using "out-of-sample" data (X_test and y_test)

In [ ]:
from sklearn.metrics import r2_score

from sklearn.metrics import mean_squared_error
# Calculate the mean_squared_error (MSE) on actual versus predicted test "y" 
# YOUR CODE HERE!
mse = mean_squared_error(
    Results["Returns"],
    Results["Predicted Return"]
)


# Using that mean-squared-error, calculate the root-mean-squared error (RMSE):
# YOUR CODE HERE!
r2 = r2_score(
    Results["Returns"],
    Results["Predicted Return"]
)

rmse = np.sqrt(mse)

print(f"Out-of-Sample Root Mean Squared Error (RMSE): {rmse}")
print(f"Out-of-Sample R^2: {r2}")


# In-Sample Performance

Evaluate the model using in-sample data (X_train and y_train)

In [ ]:
# Construct a dataframe using just the "y" training data:
# YOUR CODE HERE!

in_sample_results = Y_train.to_frame()

# Add a column of "in-sample" predictions to that dataframe:  
# YOUR CODE HERE!

in_sample_results['in_sample'] = model_1.predict(X_train)

# Calculate in-sample mean_squared_error (for comparison to out-of-sample)
# YOUR CODE HERE!

in_sample_mse = mean_squared_error(
    in_sample_results["Returns"],
    in_sample_results["in_sample"]
)


# Calculate in-sample root mean_squared_error (for comparison to out-of-sample)
# YOUR CODE HERE!


in_sample_rmse = np.sqrt(in_sample_mse)
print(f"In-sample Root Mean Squared Error (RMSE): {in_sample_rmse}")

# Conclusions

YOUR CONCLUSIONS HERE!

The RMSE is smaller in the testing data than it is in the in-sample performance.  This is a bit odd I believe. I'd imagine this is due to potential larger outliers in the training set, that could pull this value higher.  Further model evaluation would be necessary to determine why the error is lower in the testing set vs. the data the model has already seen. That said, the difference isn't all that large, so it may not matter.  Further, it does seem that this model remains ineffective, with the RMSE @ +.59% in sample and 0.415% on testing. Also, when calculating an very standard r2, we get a negative value.  This suggests that the model itself is less effective than choosing a horizontal line.  That's ungood.  Generally, this model would not appear to be tradeable in its current form. 